In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo DEBIAN_FRONTEND=noninteractive apt-get install -y cuda-9.0

--2024-11-27 02:30:34--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.nvidia.com/downloads/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb [following]
--2024-11-27 02:30:34--  https://developer.nvidia.com/downloads/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?hvt82ziE1yIayN4-dWnsXvM4bjoU-6YacUjzRAYgwGV--9W8jstxB5H-x7o2o2S7ju-V2uYBgxgGLsEtEimAT

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


In [ ]:
!pip install thundersvm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.4/507.4 kB 7.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from thundersvm import SVC

In [ ]:
# If the HPI columns have already been added
file_name = '/content/drive/MyDrive/MIE479/HPI/2020_HPI.csv'
table = pd.read_csv(file_name, low_memory=False).dropna()

In [ ]:
sampled = table.sample(n=2000000, random_state=511)

X = sampled.drop(columns=['DLQ_30_FLAG', 'DLQ_90_FLAG', 'Ongoing', 'Current_DLQ', 'Prepaid_Matured',
                          'ORIG_DTE', 'ZIP'])
y = sampled['DLQ_90_FLAG']

# CSCORE_B and CSCORE_C are highly correlated
num_col = ['ORIG_RATE', 'ORIG_AMOUNT', 'ORIG_TERM', 'OLTV', 'NUM_BO', 'DTI', 'CSCORE_B', # 'CSCORE_C',
           'NUM_UNIT', 'MI_PCT', 'HPI_ORIG_1y', 'HPI_1y_2y', 'HPI_2y_3y', 'HPI_3y_4y', 'HPI_4y_5y']
cat_col = ['FTHB_FLAG', 'PUR_Cash_out', 'PUR_Refinance', 'PUR_Purchase', 'OCC_Principal', 'OCC_Second', 'OCC_Investor']
scaler = StandardScaler()
X[num_col] = scaler.fit_transform(X[num_col])
X = pd.concat([X[num_col], X[cat_col]], axis=1)

X

,ORIG_RATE,ORIG_AMOUNT,ORIG_TERM,OLTV,NUM_BO,DTI,CSCORE_B,NUM_UNIT,MI_PCT,HPI_ORIG_1y,...,HPI_2y_3y,HPI_3y_4y,HPI_4y_5y,FTHB_FLAG,PUR_Cash_out,PUR_Refinance,PUR_Purchase,OCC_Principal,OCC_Second,OCC_Investor
662319,-0.240846,0.884771,0.583088,-0.683349,0.975570,-1.615119,0.522633,-0.11067,-0.514617,-0.624327,...,-0.026136,-0.020989,-0.562508,0,0,1,0,1,0,0
2628937,0.545242,1.457103,0.583088,-0.457697,0.975570,0.906346,-0.117948,-0.11067,-0.514617,1.238620,...,0.072324,2.252834,2.083811,0,1,0,0,1,0,0
1710354,-0.489085,2.496339,0.583088,1.403936,-0.961593,0.402053,0.670459,-0.11067,2.217099,-0.310087,...,-0.367043,0.361521,0.095012,0,0,0,1,1,0,0
994682,-0.240846,1.720677,-1.024560,-0.796175,0.975570,0.704628,0.744372,-0.11067,-0.514617,-0.680505,...,0.200781,1.287797,1.949988,0,0,1,0,1,0,0
348132,0.545242,-0.756921,0.583088,1.403936,0.975570,1.108063,0.670459,-0.11067,2.217099,-0.337317,...,-0.680934,-1.175677,-1.123364,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2573796,-1.026934,-0.523469,-1.828384,0.614153,-0.961593,-0.404816,-0.019397,-0.11067,0.031726,-0.395661,...,-1.008432,-1.164012,-1.457902,0,0,1,0,1,0,0
302436,1.310643,-0.892473,-1.828384,-0.683349,0.975570,0.200336,-0.142586,-0.11067,-0.514617,-0.003679,...,-1.071490,-0.706323,-1.537463,0,1,0,0,1,0,0
2221101,-1.792336,0.387745,0.583088,-1.134654,0.975570,-0.707392,1.015387,-0.11067,-0.514617,0.289002,...,2.884261,2.260098,0.598112,0,0,1,0,1,0,0
1812134,-0.240846,-0.011382,0.583088,-0.909001,0.975570,0.099477,0.128429,-0.11067,-0.514617,-0.315302,...,-0.015984,0.808537,2.076851,0,1,0,0,1,0,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=511)
gamma_value = 1 / (X_train.to_numpy().shape[1] * np.var(X_train.to_numpy()))

# non_linear_svm = SVC(kernel='rbf', C=1.0, gamma=gamma_value, class_weight={0: 1, 1: 33}, verbose=2)
non_linear_svm = SVC(kernel='rbf', C=1.0, gamma=gamma_value, class_weight='balanced', verbose=2)
non_linear_svm.fit(X_train, y_train)

y_pred = non_linear_svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"AUC-ROC: {round(roc_auc, 4)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7598
AUC-ROC: 0.7668
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.76      0.86    583041
           1       0.09      0.77      0.15     16959

    accuracy                           0.76    600000
   macro avg       0.54      0.77      0.51    600000
weighted avg       0.97      0.76      0.84    600000



In [ ]:
print(round(y_test.sum() / len(y_test), 4))
print(round(y_pred.sum() / len(y_pred), 4))
print(round(gamma_value, 5))

0.0283
0.2557
0.06281


In [ ]:
import pickle

with open('/content/drive/MyDrive/MIE479/HPI/thunder_svm_rbf_2020_HPI.pkl', 'wb') as file:
    pickle.dump(non_linear_svm, file)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=511)
gamma_value = 1 / (X_train.to_numpy().shape[1] * np.var(X_train.to_numpy()))

poly_svm = SVC(kernel='polynomial', C=1, gamma=gamma_value, degree=2, coef0=1, class_weight='balanced', verbose=2)
poly_svm.fit(X_train, y_train)

y_pred = poly_svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"AUC-ROC: {round(roc_auc, 4)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7241
AUC-ROC: 0.7668
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.72      0.84    583041
           1       0.08      0.81      0.14     16959

    accuracy                           0.72    600000
   macro avg       0.54      0.77      0.49    600000
weighted avg       0.97      0.72      0.82    600000



In [ ]:
print(round(y_test.sum() / len(y_test), 4))
print(round(y_pred.sum() / len(y_pred), 4))

import pickle

with open('/content/drive/MyDrive/MIE479/HPI/thunder_svm_poly2_2020_HPI.pkl', 'wb') as file:
    pickle.dump(poly_svm, file)

0.0283
0.2936


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1]
}

svm_model = SVC(kernel='rbf', class_weight={0: 1, 1: 33})

grid_search = GridSearchCV(svm_model, param_grid, cv=4, scoring='roc_auc', verbose=2)
grid_search.fit(X_train[:80000], y_train[:80000])

print("Best Parameters:", grid_search.best_params_)
print("Best AUC-ROC Score:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Fitting 4 folds for each of 16 candidates, totalling 64 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 199, in _get_response_values
    classes = estimator.classe

[CV] END .................................C=0.1, gamma=0.001; total time=   0.8s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 199, in _get_response_values
    classes = estimator.classe

[CV] END .................................C=0.1, gamma=0.001; total time=   0.8s


...

[CV] END .....................................C=100, gamma=1; total time=   1.3s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 199, in _get_response_values
    classes = estimator.classe

[CV] END .....................................C=100, gamma=1; total time=   1.3s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 199, in _get_response_values
    classes = estimator.classe

[CV] END .....................................C=100, gamma=1; total time=   1.3s
Best Parameters: {'C': 0.1, 'gamma': 0.001}
Best AUC-ROC Score: nan
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.76      0.86    145636
           1       0.09      0.76      0.15      4364

    accuracy                           0.76    150000
   macro avg       0.54      0.76      0.51    150000
weighted avg       0.96      0.76      0.84    150000



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

param_dist = {
    'C': uniform(0.1, 10),
    'gamma': uniform(0.001, 1)
}

svm_model = SVC(kernel='rbf', class_weight={0: 1, 1: 33})

random_search = RandomizedSearchCV(svm_model, param_dist, n_iter=40, cv=4, scoring='roc_auc', verbose=2, random_state=255)
random_search.fit(X_train[:80000], y_train[:80000])

print("Best Parameters:", random_search.best_params_)
print("Best AUC-ROC Score:", random_search.best_score_)

# Evaluate the best model on the test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Fitting 4 folds for each of 40 candidates, totalling 160 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 199, in _get_response_values
    classes = estimator.classe

[CV] END .....C=4.739365176859675, gamma=0.22403538373458942; total time=   2.0s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 199, in _get_response_values
    classes = estimator.classe

[CV] END .....C=4.739365176859675, gamma=0.22403538373458942; total time=   2.0s


...

[CV] END ......C=1.388726154396902, gamma=0.5925665197922657; total time=   1.0s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 199, in _get_response_values
    classes = estimator.classe

[CV] END ......C=1.388726154396902, gamma=0.5925665197922657; total time=   1.0s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 199, in _get_response_values
    classes = estimator.classe

[CV] END ......C=1.388726154396902, gamma=0.5925665197922657; total time=   1.0s
Best Parameters: {'C': 4.739365176859675, 'gamma': 0.22403538373458942}
Best AUC-ROC Score: nan
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.97    145636
           1       0.11      0.14      0.12      4364

    accuracy                           0.94    150000
   macro avg       0.54      0.55      0.55    150000
weighted avg       0.95      0.94      0.94    150000

